In [66]:
!pip install langdetect

In [67]:
import pandas as pd
from langdetect import detect
from random import sample
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

In [68]:
df = pd.read_csv('lyrics_style_classifier/lyrics.csv')
df.head()

,lyric,label
0,"Vou dar a volta no mundo eu vou, vou ver o mun...",axe
1,Nós duas Nós nuas E o nosso amor à toa Na boa ...,axe
2,Quem sabe nem era pra ser nossa Aquela lua aqu...,axe
3,Já são cinco da manhã E não dormi quase nada S...,axe
4,Amor de verdade eu só senti Foi com você meu b...,axe


In [69]:
def cat_to_num(label):
    if label == 'axe':
        label = 0
    elif label == 'sertanejo':
        label = 1
    elif label == 'gospelreligioso':
        label = 2
    elif label == 'bossa-nova':
        label = 3
    elif label == 'forro':
        label = 4
    elif label == 'mpb':
        label = 5
    elif label == 'samba':
        label = 6

    return label



In [70]:
df['label'] = df['label'].apply(lambda l: cat_to_num(l))
df.dtypes


lyric    object
label     int64
dtype: object

Distribuição de textos por categoria

In [71]:
df.label.value_counts()

0    907
5    899
1    897
3    896
4    895
2    894
6    893
Name: label, dtype: int64

Detecta letras com linguagem diferente de português e remove do dataset

In [72]:
not_pt = []
for lyric, index in zip(df.lyric, df.index):
    try:
        if len(lyric)<=0:
            not_pt.append(index)
        elif(detect(lyric) != 'pt'):
            not_pt.append(index)
    except Exception:
        not_pt.append(index)

print(f"Foram removidos {len(not_pt)} textos com linguagem diferente de português")

print("Amostras de textos excluídos:")
print('\n'.join([f'{text[:100]}...' for text in df.iloc[sample(not_pt, 3)].lyric]))

df.drop(not_pt, inplace=True)

Foram removidos 284 textos com linguagem diferente de português
Amostras de textos excluídos:
Sora o oshi agete Te o nobasu kimi gogatsu no koto Dou ka kite hoshii Mizugiwa made kite hoshii Tsub...
Vagando paso la vida nomas recorriendo el mundo si quieren que se los digan yo soy un alma sin dueño...
May Day! May Day! [SOS!] May Day! May Day! [SOS!] Kokoro o Tsuki sasu Hishi no Himeki Saa Iku ze Ore...


Distribuição de textos por categoria após as remoções

In [73]:
df.label.value_counts()

1    895
4    891
5    889
6    886
2    874
3    799
0    763
Name: label, dtype: int64

# Algoritmos de Machine Learning

Aplica alguns algoritmos de ML para obter uma baseline para o desenvolvimento.

Separa o dataset com os conjuntos para treino e teste

In [74]:
sentences = df.lyric.values
y = df.label.values

sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)

Aplica transformação de textos para números

In [75]:
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

x_train = vectorizer.transform(sentences_train)
x_test = vectorizer.transform(sentences_test)

x_train[0]

<1x25324 sparse matrix of type '<class 'numpy.int64'>'
	with 67 stored elements in Compressed Sparse Row format>

In [84]:
def show_model_classification_score(model_obj, name):
    model = model_obj.fit(x_train, y_train)
    score = model.score(x_test, y_test)

    print(f"Mean accuracy for model {name}: {score}")

Usa algortimos baseados em árvore e sem tuning para obter acurácia base

In [85]:
show_model_classification_score(model_obj=DecisionTreeClassifier(), name="Decision Tree")
show_model_classification_score(model_obj=RandomForestClassifier(n_estimators=20, random_state=0), name="Random Forest")
show_model_classification_score(model_obj=XGBClassifier(), name="XGBoost")

Score for model Decision Tree: 0.388
Score for model Random Forest: 0.43533333333333335
Score for model XGBoost: 0.5126666666666667
